<div align="center"> 
    <hr>
  <font size="6"><b>Dev | Data Processing Notebook</b> </font>
   <hr>
</div>

In [1]:
!python --version

Python 3.9.16


In [2]:
import os
import sys
import pandas as pd
import networkx
from networkx import draw, Graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pyjedai.utils import (
    text_cleaning_method,
    print_clusters,
    print_blocks,
    print_candidate_pairs
)
from pyjedai.evaluation import Evaluation
from pyjedai.datamodel import Data
d1 = pd.read_csv("../data/ccer/D2/abt.csv", sep='|', engine='python')
d2 = pd.read_csv("../data/ccer/D2/buy.csv", sep='|', engine='python')
gt = pd.read_csv("../data/ccer/D2/gt.csv", sep='|', engine='python')

data = Data(
    dataset_1=d1,
    attributes_1=['id','name','description'],
    id_column_name_1='id',
    dataset_2=d2,
    attributes_2=['id','name','description'],
    id_column_name_2='id',
    ground_truth=gt,
)

[nltk_data] Downloading package stopwords to /home/jm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data.dataset_2.iloc[216]

id                                      216
name           LG Navy Blue SteamWasher 27'
description                                
price                                      
Name: 216, dtype: object

In [4]:
data.print_specs()

------------------------- Data -------------------------
Type of Entity Resolution:  Clean-Clean
Dataset-1:
	Number of entities:  1076
	Number of NaN values:  0
	Attributes: 
		 ['id', 'name', 'description']
Dataset-2:
	Number of entities:  1076
	Number of NaN values:  0
	Attributes: 
		 ['id', 'name', 'description']

Total number of entities:  2152
Number of matching pairs in ground-truth:  1076
-------------------------------------------------------- 



In [5]:
data.dataset_1

,id,name,description,price
0,0,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ ...,
1,1,Bose Acoustimass 5 Series III Speaker System -...,Bose Acoustimass 5 Series III Speaker System -...,399.0
2,2,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecti...,49.0
3,3,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...,
4,4,Bose 27028 161 Bookshelf Pair Speakers In Whit...,Bose 161 Bookshelf Speakers In White - 161WH/ ...,158.0
...,...,...,...,...
1071,1071,Logitech Cordless Desktop Wave Keyboard And Mo...,Logitech Cordless Desktop Wave Keyboard And Mo...,79.0
1072,1072,Mitsubishi DLP Black TV Stand - MBS73V,Mitsubishi DLP Black TV Stand - MBS73V/ Matchi...,549.0
1073,1073,Logitech Digital Precision PC Gaming Headset -...,Logitech Digital Precision PC Gaming Headset -...,49.0
1074,1074,Logitech 2.1 Multimedia Silver Speaker System ...,Logitech 2.1 Multimedia Silver Speaker System ...,


In [6]:
data.dataset_2

,id,name,description,price
0,0,Linksys EtherFast EZXS88W Ethernet Switch - EZ...,Linksys EtherFast 8-Port 10/100 Switch (New/Wo...,
1,1,Linksys EtherFast EZXS55W Ethernet Switch,5 x 10/100Base-TX LAN,
2,2,Netgear ProSafe FS105 Ethernet Switch - FS105NA,NETGEAR FS105 Prosafe 5 Port 10/100 Desktop Sw...,
3,3,Belkin Pro Series High Integrity VGA/SVGA Moni...,1 x HD-15 - 1 x HD-15 - 10ft - Beige,
4,4,Netgear ProSafe JFS516 Ethernet Switch,Netgear ProSafe 16 Port 10/100 Rackmount Switc...,
...,...,...,...,...
1071,1071,Sony VAIO FW378J/B Notebook - VGNFW378J/B,Intel Centrino 2 Core 2 Duo P8600 2.4GHz - 16....,
1072,1072,Sennheiser CX380 Sennheiser CX 380 Sport II Gr...,,
1073,1073,IWORK 09 RETAIL-INT DVD - MB942Z/A,,
1074,1074,IWORK 09 FAMILY PACK-INT DVD - MB943Z/A,,


In [7]:
data.clean_dataset(remove_stopwords = True, 
                   remove_punctuation = False,
                   remove_numbers = False,
                   remove_unicodes = False)

In [8]:
data.dataset_1

,id,name,description,price
0,0,sony turntable - pslx350h,sony turntable - pslx350h/ belt drive system/ ...,
1,1,bose acoustimass 5 series iii speaker system -...,bose acoustimass 5 series iii speaker system -...,399.0
2,2,sony switcher - sbv40s,sony switcher - sbv40s/ eliminates disconnecti...,49.0
3,3,sony 5 disc cd player - cdpce375,sony 5 disc cd player- cdpce375/ 5 disc change...,
4,4,bose 27028 161 bookshelf pair speakers white -...,bose 161 bookshelf speakers white - 161wh/ art...,158.0
...,...,...,...,...
1071,1071,logitech cordless desktop wave keyboard mouse ...,logitech cordless desktop wave keyboard mouse ...,79.0
1072,1072,mitsubishi dlp black tv stand - mbs73v,mitsubishi dlp black tv stand - mbs73v/ matchi...,549.0
1073,1073,logitech digital precision pc gaming headset -...,logitech digital precision pc gaming headset -...,49.0
1074,1074,logitech 2.1 multimedia silver speaker system ...,logitech 2.1 multimedia silver speaker system ...,


In [9]:
data.dataset_2

,id,name,description,price
0,0,linksys etherfast ezxs88w ethernet switch - ez...,linksys etherfast 8-port 10/100 switch (new/wo...,
1,1,linksys etherfast ezxs55w ethernet switch,5 x 10/100base-tx lan,
2,2,netgear prosafe fs105 ethernet switch - fs105na,netgear fs105 prosafe 5 port 10/100 desktop sw...,
3,3,belkin pro series high integrity vga/svga moni...,1 x hd-15 - 1 x hd-15 - 10ft - beige,
4,4,netgear prosafe jfs516 ethernet switch,netgear prosafe 16 port 10/100 rackmount switc...,
...,...,...,...,...
1071,1071,sony vaio fw378j/b notebook - vgnfw378j/b,intel centrino 2 core 2 duo p8600 2.4ghz - 16....,
1072,1072,sennheiser cx380 sennheiser cx 380 sport ii gr...,,
1073,1073,iwork 09 retail-int dvd - mb942z/a,,
1074,1074,iwork 09 family pack-int dvd - mb943z/a,,


In [10]:
data.clean_dataset(remove_stopwords = False, 
                   remove_punctuation = True, 
                   remove_numbers = False,
                   remove_unicodes = False)

In [11]:
data.dataset_1

,id,name,description,price
0,0,sony turntable pslx350h,sony turntable pslx350h belt drive system 331...,
1,1,bose acoustimass 5 series iii speaker system ...,bose acoustimass 5 series iii speaker system ...,3990
2,2,sony switcher sbv40s,sony switcher sbv40s eliminates disconnecting...,490
3,3,sony 5 disc cd player cdpce375,sony 5 disc cd player cdpce375 5 disc changer ...,
4,4,bose 27028 161 bookshelf pair speakers white ...,bose 161 bookshelf speakers white 161wh artic...,1580
...,...,...,...,...
1071,1071,logitech cordless desktop wave keyboard mouse ...,logitech cordless desktop wave keyboard mouse ...,790
1072,1072,mitsubishi dlp black tv stand mbs73v,mitsubishi dlp black tv stand mbs73v matching...,5490
1073,1073,logitech digital precision pc gaming headset ...,logitech digital precision pc gaming headset ...,490
1074,1074,logitech 21 multimedia silver speaker system ...,logitech 21 multimedia silver speaker system ...,


In [12]:
data.dataset_2

,id,name,description,price
0,0,linksys etherfast ezxs88w ethernet switch ezx...,linksys etherfast 8port 10100 switch newworkgroup,
1,1,linksys etherfast ezxs55w ethernet switch,5 x 10100basetx lan,
2,2,netgear prosafe fs105 ethernet switch fs105na,netgear fs105 prosafe 5 port 10100 desktop switch,
3,3,belkin pro series high integrity vgasvga monit...,1 x hd15 1 x hd15 10ft beige,
4,4,netgear prosafe jfs516 ethernet switch,netgear prosafe 16 port 10100 rackmount switch...,
...,...,...,...,...
1071,1071,sony vaio fw378jb notebook vgnfw378jb,intel centrino 2 core 2 duo p8600 24ghz 164 ...,
1072,1072,sennheiser cx380 sennheiser cx 380 sport ii gr...,,
1073,1073,iwork 09 retailint dvd mb942za,,
1074,1074,iwork 09 family packint dvd mb943za,,


In [13]:
data.clean_dataset(remove_stopwords = False, 
                   remove_punctuation = False, 
                   remove_numbers = True,
                   remove_unicodes = False)

In [14]:
data.dataset_1

,id,name,description,price
0,,sony turntable pslxh,sony turntable pslxh belt drive system rpm ...,
1,,bose acoustimass series iii speaker system ambk,bose acoustimass series iii speaker system a...,
2,,sony switcher sbvs,sony switcher sbvs eliminates disconnecting r...,
3,,sony disc cd player cdpce,sony disc cd player cdpce disc changer varia...,
4,,bose bookshelf pair speakers white wh,bose bookshelf speakers white wh articulated...,
...,...,...,...,...
1071,,logitech cordless desktop wave keyboard mouse ...,logitech cordless desktop wave keyboard mouse ...,
1072,,mitsubishi dlp black tv stand mbsv,mitsubishi dlp black tv stand mbsv matching b...,
1073,,logitech digital precision pc gaming headset,logitech digital precision pc gaming headset ...,
1074,,logitech multimedia silver speaker system z,logitech multimedia silver speaker system z ...,


In [15]:
data.dataset_2

,id,name,description,price
0,,linksys etherfast ezxsw ethernet switch ezxsw,linksys etherfast port switch newworkgroup,
1,,linksys etherfast ezxsw ethernet switch,x basetx lan,
2,,netgear prosafe fs ethernet switch fsna,netgear fs prosafe port desktop switch,
3,,belkin pro series high integrity vgasvga monit...,x hd x hd ft beige,
4,,netgear prosafe jfs ethernet switch,netgear prosafe port rackmount switch jfsna,
...,...,...,...,...
1071,,sony vaio fwjb notebook vgnfwjb,intel centrino core duo p ghz gb ddr sdra...,
1072,,sennheiser cx sennheiser cx sport ii greyorange,,
1073,,iwork retailint dvd mbza,,
1074,,iwork family packint dvd mbza,,


In [16]:
data.clean_dataset(remove_stopwords = False, 
                   remove_punctuation = False, 
                   remove_numbers = False,
                   remove_unicodes = True)

In [17]:
data.dataset_1

,id,name,description,price
0,,sony turntable pslxh,sony turntable pslxh belt drive system rpm ...,
1,,bose acoustimass series iii speaker system ambk,bose acoustimass series iii speaker system a...,
2,,sony switcher sbvs,sony switcher sbvs eliminates disconnecting r...,
3,,sony disc cd player cdpce,sony disc cd player cdpce disc changer varia...,
4,,bose bookshelf pair speakers white wh,bose bookshelf speakers white wh articulated...,
...,...,...,...,...
1071,,logitech cordless desktop wave keyboard mouse ...,logitech cordless desktop wave keyboard mouse ...,
1072,,mitsubishi dlp black tv stand mbsv,mitsubishi dlp black tv stand mbsv matching b...,
1073,,logitech digital precision pc gaming headset,logitech digital precision pc gaming headset ...,
1074,,logitech multimedia silver speaker system z,logitech multimedia silver speaker system z ...,


In [18]:
data.dataset_2

,id,name,description,price
0,,linksys etherfast ezxsw ethernet switch ezxsw,linksys etherfast port switch newworkgroup,
1,,linksys etherfast ezxsw ethernet switch,x basetx lan,
2,,netgear prosafe fs ethernet switch fsna,netgear fs prosafe port desktop switch,
3,,belkin pro series high integrity vgasvga monit...,x hd x hd ft beige,
4,,netgear prosafe jfs ethernet switch,netgear prosafe port rackmount switch jfsna,
...,...,...,...,...
1071,,sony vaio fwjb notebook vgnfwjb,intel centrino core duo p ghz gb ddr sdra...,
1072,,sennheiser cx sennheiser cx sport ii greyorange,,
1073,,iwork retailint dvd mbza,,
1074,,iwork family packint dvd mbza,,


---

# All and experiment 

In [19]:
data.clean_dataset(remove_stopwords = True, 
                   remove_punctuation = True, 
                   remove_numbers = True,
                   remove_unicodes = True)

In [20]:
data.dataset_1

,id,name,description,price
0,,sony turntable pslxh,sony turntable pslxh belt drive system rpm spe...,
1,,bose acoustimass series iii speaker system ambk,bose acoustimass series iii speaker system amb...,
2,,sony switcher sbvs,sony switcher sbvs eliminates disconnecting re...,
3,,sony disc cd player cdpce,sony disc cd player cdpce disc changer variabl...,
4,,bose bookshelf pair speakers white wh,bose bookshelf speakers white wh articulated a...,
...,...,...,...,...
1071,,logitech cordless desktop wave keyboard mouse ...,logitech cordless desktop wave keyboard mouse ...,
1072,,mitsubishi dlp black tv stand mbsv,mitsubishi dlp black tv stand mbsv matching ba...,
1073,,logitech digital precision pc gaming headset,logitech digital precision pc gaming headset c...,
1074,,logitech multimedia silver speaker system z,logitech multimedia silver speaker system z th...,


In [21]:
data.dataset_2

,id,name,description,price
0,,linksys etherfast ezxsw ethernet switch ezxsw,linksys etherfast port switch newworkgroup,
1,,linksys etherfast ezxsw ethernet switch,x basetx lan,
2,,netgear prosafe fs ethernet switch fsna,netgear fs prosafe port desktop switch,
3,,belkin pro series high integrity vgasvga monit...,x hd x hd ft beige,
4,,netgear prosafe jfs ethernet switch,netgear prosafe port rackmount switch jfsna,
...,...,...,...,...
1071,,sony vaio fwjb notebook vgnfwjb,intel centrino core duo p ghz gb ddr sdram gb ...,
1072,,sennheiser cx sennheiser cx sport ii greyorange,,
1073,,iwork retailint dvd mbza,,
1074,,iwork family packint dvd mbza,,


In [22]:
from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

emb = EmbeddingsNNBlockBuilding(vectorizer='sminilm',
                                similarity_search='faiss')
blocks, g = emb.build_blocks(data, 
                          top_k=5,
                            similarity_distance='cosine',
                          load_embeddings_if_exist=False,
                          save_embeddings=False,
                          with_entity_matching=True)

from pyjedai.clustering import UniqueMappingClustering

ccc = UniqueMappingClustering()
clusters = ccc.process(g, data, similarity_threshold=0.7)
ccc.evaluate(clusters, with_classification_report=True)

/home/conda/miniconda3/envs/pyjedai-progressive/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Building blocks via Embeddings-NN Block Building [sminilm, faiss]


Embeddings-NN Block Building [sminilm, faiss]:   0%|          | 0/2152 [00:00<?, ?it/s]

Device selected:  cuda


Embeddings-NN Block Building [sminilm, faiss]: 100%|██████████| 2152/2152 [00:23<00:00, 92.37it/s] 

***************************************************************************************************************************
                                         Μethod:  Unique Mapping Clustering
***************************************************************************************************************************
Method name: Unique Mapping Clustering
Parameters: 
Runtime: 0.0277 seconds
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Performance:
	Precision:     69.00% 
	Recall:        47.58%
	F1-score:      56.33%
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Classification report:
	True positives: 512
	False positives: 230
	True negatives: 1156136
	False negatives: 564
	Total comparisons: 742
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


{'Precision %': 69.00269541778977,
 'Recall %': 47.58364312267658,
 'F1 %': 56.32563256325632,
 'True Positives': 512,
 'False Positives': 230,
 'True Negatives': 1156136,
 'False Negatives': 564}